In [38]:
import pandas as pd
import requests
import json
import time
import os

from dotenv import load_dotenv
from datetime import datetime

In [39]:
load_dotenv()

True

In [40]:
def nyt_search(start_date, end_date, key, pages=1, t_sleep=1, q=None, news_desk=None):
    responses = []
    base_url = "https://api.nytimes.com/svc/search/v2/articlesearch"
    
    for i in range(pages):
        url = base_url + '.json?fq=' + news_desk + '&facet_field=day_of_week&facet=true&page=' + str(i) + '&begin_date=' + start_date + '&end_date=' + end_date + '&api-key=' + key
        data = requests.get(url).json()
        responses.append(data)
        time.sleep(t_sleep)
    
    return(responses)

def nyt_search_responses_unwrap(responses):
    
    result = {
        "Date" : [],
        "Source" : [],
        "Headline" : [],
        "Lead Paragraph" : []
    }
    
    for d in responses:
        for article in d['response']['docs']:
            result["Date"].append(article['pub_date'][:10])
            result["Source"].append(article['source'])
            result["Headline"].append(article['headline']['main'])
            result["Lead Paragraph"].append(article['lead_paragraph'])
            
    return(result)
    
    

In [30]:
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch"
news_desk = "news_desk:(\"Business\" \"Politics\" \"Financial\" \"Foreign\" \"Jobs\" \"Personal Investing\")"

In [31]:
oct_responses = nyt_search(start_date='20201015', end_date='20201114', key=os.getenv("NYT_API_KEY"), pages=10, news_desk=news_desk)

In [31]:
nov_responses = nyt_search(start_date='20201115', end_date='20201214', key=os.getenv("NYT_API_KEY"), pages=10, news_desk=news_desk)

In [31]:
dec_responses = nyt_search(start_date='20201215', end_date='20210114', key=os.getenv("NYT_API_KEY"), pages=10, news_desk=news_desk)

In [61]:
oct_dict = nyt_search_responses_unwrap(oct_responses)

In [62]:
df = pd.DataFrame.from_dict(oct_dict)

In [63]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")
df = df.sort_index(axis=0)

In [76]:
srs = df.groupby(['Date'])['Headline'].apply(','.join).reset_index()

In [79]:
srs["Headline"][0]

'Stressed About the Election? Don’t Take It Out on Your Investments,Advertisers Flock to Election Night, When Live TV Is the Main Event Again,Women Converge on Warsaw, Heightening Poland’s Largest Protests in Decades,Qatar Apologizes for Airport Strip Searches and Pursues Charges,Europe Risks a New Economic Downturn as Lockdowns Return'